In [34]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine_similarity(a, b):
    a = np.atleast_2d(a)
    b = np.atleast_2d(b)

    if a.shape[0] == 1 and b.shape[0] > 1:
        a = a.T  # ensure shape is (n, 1) if needed
    if b.shape[0] == 1 and a.shape[0] > 1:
        b = b.T

    return cosine_similarity(a, b)

In [35]:
# Define the folder containing the gradient results
gradient_results_folder = "gradient_results"

# Iterate through all subfolders in the gradient_results folder
for subfolder in os.listdir(gradient_results_folder):
    subfolder_path = os.path.join(gradient_results_folder, subfolder)
    
    # Initialize a list to store the summary data
    summary_data = []

    if os.path.isdir(subfolder_path) and subfolder.startswith("results_gradients_"):
        # Extract sample_size, num_classes, and learning_rate from the folder name
        _, _, sample_size, num_classes, learning_rate = subfolder.split("_")
        
        # Initialize dictionaries to store gradients for each parameter and bias/weight
        gradients_dict = {}

        # Iterate through class folders (0, 1, 2, ..., num_classes-1)
        for class_folder in os.listdir(subfolder_path):
            class_folder_path = os.path.join(subfolder_path, class_folder)
            if os.path.isdir(class_folder_path):
                # Iterate through gradient files in the class folder
                for gradient_file in os.listdir(class_folder_path):
                    if gradient_file.startswith("gradients_cycle_") and gradient_file.endswith(".npy"):
                        # Extract param_name and bias/weight from the file name
                        _, _, cycle, _, param_name_bias_weight = gradient_file.split("_")
                        param_name, bias_weight, _ = param_name_bias_weight.rsplit(".")
                        
                        # Load the gradient data
                        gradient_path = os.path.join(class_folder_path, gradient_file)
                        gradients = np.load(gradient_path)
                        
                        # Store gradients in the dictionary
                        key = (param_name, bias_weight)
                        if key not in gradients_dict:
                            gradients_dict[key] = []
                        gradients_dict[key].append(gradients)
                        
        # Calculate statistics across all gradients for each parameter and bias/weight
        for (param_name, bias_weight), gradients in gradients_dict.items():
            gradients = np.array(gradients)
            avg = np.mean(gradients, axis=0)
            min_val = np.min(gradients, axis=0)
            max_val = np.max(gradients, axis=0)
            std = np.std(gradients, axis=0)
            
            # Initialize a list to store the cosine similarity data
            cosine_similarity_data = []
            
            for gradient in gradients:
                c_sim = compute_cosine_similarity(gradient, avg)
                cosine_similarity_data.append(c_sim)
            # Append the data to the summary list
            summary_data.append([
            sample_size, num_classes, learning_rate, param_name, bias_weight, avg, min_val, max_val, std, cosine_similarity_data
            ])
            
        # Create a DataFrame from the summary data
        summary_df = pd.DataFrame(summary_data, columns=[
            "sample_size", "num_classes", "learning_rate", "param_name", "bias/weight", "avg", "min", "max", "std", "cosine_similarity_data"
        ])

        # Save the DataFrame to a CSV file with a name involving sample_size, num_classes, and learning_rate
        summary_csv_name = f"gradients_summary_{sample_size}_{num_classes}_{learning_rate}.csv"
        summary_csv_path = os.path.join(gradient_results_folder, summary_csv_name)
        summary_df.to_csv(summary_csv_path, index=False)